In [1]:
import tensorflow.compat.v1 as tf1
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "2"
config = tf1.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.9
session = tf1.Session(config=config)

2024-12-21 19:15:57.190866: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-21 19:15:57.237281: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-21 19:15:58.048739: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-12-21 19:15:59.217368: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 36305 MB memory:  -> device: 0, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:86:00.0, compute capab

In [2]:
import os
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torch.nn.functional as F
from tqdm import tqdm

# PSNR 계산 함수
def calculate_psnr(img1, img2, max_val=1.0):
    mse = F.mse_loss(img1, img2, reduction='mean')
    if mse == 0:
        return float('inf')
    return 10 * torch.log10(max_val**2 / mse)

# SSIM 계산 함수
def calculate_ssim(img1, img2, max_val=1.0):
    C1 = (0.01 * max_val) ** 2
    C2 = (0.03 * max_val) ** 2

    mu1 = F.avg_pool2d(img1, kernel_size=3, stride=1, padding=1)
    mu2 = F.avg_pool2d(img2, kernel_size=3, stride=1, padding=1)

    sigma1_sq = F.avg_pool2d(img1 * img1, kernel_size=3, stride=1, padding=1) - mu1.pow(2)
    sigma2_sq = F.avg_pool2d(img2 * img2, kernel_size=3, stride=1, padding=1) - mu2.pow(2)
    sigma12 = F.avg_pool2d(img1 * img2, kernel_size=3, stride=1, padding=1) - mu1 * mu2

    ssim_map = ((2 * mu1 * mu2 + C1) * (2 * sigma12 + C2)) / ((mu1.pow(2) + mu2.pow(2) + C1) * (sigma1_sq + sigma2_sq + C2))
    return ssim_map.mean()

# 저해상도 이미지 생성 함수
def create_low_res_images(input_dir, output_dir, scale):
    os.makedirs(output_dir, exist_ok=True)
    for filename in os.listdir(input_dir):
        if filename.lower().endswith(('png', 'jpg', 'jpeg')):
            img = Image.open(os.path.join(input_dir, filename))
            lr_img = img.resize((img.width // scale, img.height // scale), Image.Resampling.BICUBIC)
            lr_img.save(os.path.join(output_dir, filename))

# 데이터셋 클래스
def get_image_paths(directory):
    return sorted([os.path.join(directory, f) for f in os.listdir(directory) if f.lower().endswith(('png', 'jpg', 'jpeg'))])

class SRDataset(Dataset):
    def __init__(self, hr_dir, lr_dir, transform=None, input_size=(64, 64), scale_factor=4):
        self.hr_paths = get_image_paths(hr_dir)
        self.lr_paths = get_image_paths(lr_dir)
        self.transform = transform
        self.input_size = input_size
        self.target_size = (input_size[0] * scale_factor, input_size[1] * scale_factor)

    def __len__(self):
        return len(self.hr_paths)

    def __getitem__(self, idx):
        hr_img = Image.open(self.hr_paths[idx]).convert("RGB")
        lr_img = Image.open(self.lr_paths[idx]).convert("RGB")

        lr_img = lr_img.resize(self.input_size, Image.Resampling.BICUBIC)
        hr_img = hr_img.resize(self.target_size, Image.Resampling.BICUBIC)

        if self.transform:
            hr_img = self.transform(hr_img)
            lr_img = self.transform(lr_img)

        return lr_img, hr_img

# RCAN 모델 클래스
class RCAN(nn.Module):
    def __init__(self, scale_factor=4, num_channels=64, num_blocks=10, num_groups=5):
        super(RCAN, self).__init__()
        self.head = nn.Conv2d(3, num_channels, kernel_size=3, padding=1)

        self.body = nn.Sequential(
            *[ResidualGroup(num_channels, num_blocks) for _ in range(num_groups)]
        )

        self.tail = nn.Sequential(
            nn.Conv2d(num_channels, num_channels * (scale_factor ** 2), kernel_size=3, padding=1),
            nn.PixelShuffle(scale_factor),
            nn.Conv2d(num_channels, 3, kernel_size=3, padding=1)
        )

    def forward(self, x):
        x = self.head(x)
        res = self.body(x)
        x = res + x
        x = self.tail(x)
        return x

class ResidualGroup(nn.Module):
    def __init__(self, num_channels, num_blocks):
        super(ResidualGroup, self).__init__()
        self.blocks = nn.Sequential(
            *[ResidualBlock(num_channels) for _ in range(num_blocks)]
        )
        self.conv = nn.Conv2d(num_channels, num_channels, kernel_size=3, padding=1)

    def forward(self, x):
        res = self.blocks(x)
        return res + x

class ResidualBlock(nn.Module):
    def __init__(self, num_channels):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(num_channels, num_channels, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(num_channels, num_channels, kernel_size=3, padding=1)
        self.relu = nn.ReLU(inplace=True)
        self.ca = ChannelAttention(num_channels)

    def forward(self, x):
        res = self.relu(self.conv1(x))
        res = self.conv2(res)
        res = self.ca(res)
        return res + x

class ChannelAttention(nn.Module):
    def __init__(self, num_channels, reduction=16):
        super(ChannelAttention, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
            nn.Linear(num_channels, num_channels // reduction, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(num_channels // reduction, num_channels, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        b, c, _, _ = x.size()
        y = self.avg_pool(x).view(b, c)
        y = self.fc(y).view(b, c, 1, 1)
        return x * y

# 학습 및 검증 루프 함수
def train_model(model, train_loader, valid_loader, criterion, optimizer, device, num_epochs, save_path, patience=5):
    best_valid_loss = float('inf')
    early_stop_counter = 0

    for epoch in range(num_epochs):
        model.train()
        train_loss, train_psnr = 0, 0

        for lr_imgs, hr_imgs in tqdm(train_loader, desc=f"Training Epoch {epoch + 1}/{num_epochs}"):
            lr_imgs, hr_imgs = lr_imgs.to(device), hr_imgs.to(device)
            preds = model(lr_imgs)
            loss = criterion(preds, hr_imgs)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            train_psnr += calculate_psnr(preds, hr_imgs).item()

        train_loss /= len(train_loader)
        train_psnr /= len(train_loader)

        model.eval()
        valid_loss, valid_psnr, valid_ssim = 0, 0, 0

        with torch.no_grad():
            for lr_imgs, hr_imgs in tqdm(valid_loader, desc="Validating"):
                lr_imgs, hr_imgs = lr_imgs.to(device), hr_imgs.to(device)
                preds = model(lr_imgs)
                valid_loss += criterion(preds, hr_imgs).item()
                valid_psnr += calculate_psnr(preds, hr_imgs).item()
                valid_ssim += calculate_ssim(preds, hr_imgs).item()

        valid_loss /= len(valid_loader)
        valid_psnr /= len(valid_loader)
        valid_ssim /= len(valid_loader)

        print(f"Epoch {epoch + 1}/{num_epochs} | Train Loss: {train_loss:.4f} | Train PSNR: {train_psnr:.4f} | "
              f"Valid Loss: {valid_loss:.4f} | Valid PSNR: {valid_psnr:.4f} | Valid SSIM: {valid_ssim:.4f}")

        if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
            torch.save(model.state_dict(), save_path)
            print("Best model saved!")
            early_stop_counter = 0
        else:
            early_stop_counter += 1

        if early_stop_counter >= patience:
            print("Early stopping triggered!")
            break

# 설정 및 초기화
def main():
    train_hr_dir = "/home/a202192006/image/train"
    train_lr_dir = "/home/a202192006/image/train_lr"
    valid_hr_dir = "/home/a202192006/image/valid"
    valid_lr_dir = "/home/a202192006/image/valid_lr"

    # 저해상도 데이터셋 생성
    create_low_res_images(train_hr_dir, train_lr_dir, scale=4)
    create_low_res_images(valid_hr_dir, valid_lr_dir, scale=4)

    transform = transforms.ToTensor()
    train_dataset = SRDataset(train_hr_dir, train_lr_dir, transform, input_size=(64, 64), scale_factor=4)
    valid_dataset = SRDataset(valid_hr_dir, valid_lr_dir, transform, input_size=(64, 64), scale_factor=4)

    train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=4, pin_memory=True)
    valid_loader = DataLoader(valid_dataset, batch_size=4, shuffle=False, num_workers=4, pin_memory=True)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # 모델 및 학습 설정
    model = RCAN().to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-4)
    save_path = "/home/a202192006/image/Earlystopping/RCAN_Earlystopping_best_model.pth"

    print("Training RCAN...")
    train_model(model, train_loader, valid_loader, criterion, optimizer, device, num_epochs=100, save_path=save_path, patience=5)

    # 테스트 루프
    test_dataset = SRDataset(valid_hr_dir, valid_lr_dir, transform, input_size=(64, 64), scale_factor=4)
    test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

    model.load_state_dict(torch.load(save_path))
    model.eval()

    test_psnr, test_ssim = 0, 0
    with torch.no_grad():
        for lr_imgs, hr_imgs in tqdm(test_loader, desc="Testing"):
            lr_imgs, hr_imgs = lr_imgs.to(device), hr_imgs.to(device)
            preds = model(lr_imgs)
            test_psnr += calculate_psnr(preds, hr_imgs).item()
            test_ssim += calculate_ssim(preds, hr_imgs).item()

    test_psnr /= len(test_loader)
    test_ssim /= len(test_loader)

    print(f"RCAN - Test PSNR: {test_psnr:.4f}, Test SSIM: {test_ssim:.4f}")

if __name__ == "__main__":
    main()


/home/a202192006/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Training RCAN...


Validating:   0%|                                                                                                                                               | 0/1750 [00:00<?, ?it/s][W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
Validating: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1750/1750 [00:38<00:00, 45.55it/s]


Epoch 1/100 | Train Loss: 0.0089 | Train PSNR: 21.7850 | Valid Loss: 0.0058 | Valid PSNR: 22.9686 | Valid SSIM: 0.7502
Best model saved!


Training Epoch 2/100:   0%|                                                                                                                                    | 0/17500 [00:00<?, ?it/s][W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
Validating:   0%|                                                                                                                                               | 0/1750 [00:00<?, ?it/s][W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe

...

Epoch 22/100 | Train Loss: 0.0031 | Train PSNR: 25.4314 | Valid Loss: 0.0035 | Valid PSNR: 25.1595 | Valid SSIM: 0.8148


Training Epoch 23/100:   0%|                                                                                                                                   | 0/17500 [00:00<?, ?it/s][W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
Validating:   0%|                                                                                                                                               | 0/1750 [00:00<?, ?it/s][W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe

Epoch 23/100 | Train Loss: 0.0031 | Train PSNR: 25.4569 | Valid Loss: 0.0035 | Valid PSNR: 25.1914 | Valid SSIM: 0.8153
Early stopping triggered!


Testing: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7000/7000 [02:10<00:00, 53.75it/s]

RCAN - Test PSNR: 26.3258, Test SSIM: 0.8150
